# Notes

- Semua nama kolom kini snake_case (mis. `user_id`, `stress_level`, `created_at`, `study_hour_per_day`).
- Kolom `is_restored` adalah metadata input/restore dan **tidak** dipakai sebagai fitur model.


In [2]:
# =====================================================================================
# PERSONALIZED_FORECAST (TRUE Personalized, per-user model) - 1 CELL (SVM Calibration FIXED)
#
# Tujuan:
# - TRUE PERSONALIZED: 1 model per user (latih terpisah per user).
# - user_id TIDAK dipakai sebagai fitur (default), hanya untuk grouping & split.
# - Target binary: y=1 jika stress_level_pred>=1, else 0.
#
# Baselines:
# - L1 Persistence: y(t)=y(t-1)
# - L2 Markov per-user: P(high_t | prev_high, dow) + threshold tuning via pooled per-user CV
#
# Model candidates (per-user):
# - LogisticRegression
# - DecisionTree
# - RandomForest
# - ExtraTrees
# - HistGradientBoosting
# - GradientBoosting
# - AdaBoost
# - BaggingTree
# - LinearSVC + CalibratedClassifierCV (SAFE: cv adaptif per-fold, skip fold jika cv_k<2,
#   dan skip kandidat SVM seluruhnya jika ada user tidak feasible / tidak ada fold valid)
#
# Split:
# - per user time-based
# - TEST = last TEST_LEN
# - CV folds = windows di train_pool masing-masing user
#
# Output:
# - ../models/personalized_forecast.joblib
# =====================================================================================

import numpy as np
import pandas as pd
from pathlib import Path
import joblib

from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import ParameterGrid

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (
    RandomForestClassifier, ExtraTreesClassifier,
    HistGradientBoostingClassifier,
    GradientBoostingClassifier, AdaBoostClassifier,
    BaggingClassifier
)
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV


# =========================
# 0) CONFIG
# =========================
CANDIDATE_PATHS = [
    Path("../datasets/global_dataset_pred.csv"),
]
DATA_PATH = next((p for p in CANDIDATE_PATHS if p.exists()), None)
if DATA_PATH is None:
    raise FileNotFoundError("global_dataset_pred.csv tidak ditemukan. Cek CANDIDATE_PATHS / DATA_PATH.")

MODEL_OUT = Path("../models/personalized_forecast.joblib")

DATE_COL   = "date"
USER_COL   = "user_id"
TARGET_COL = "stress_level_pred"  # 0..2

WINDOW   = 3
TEST_LEN = 12

VAL_WINDOWS = [(10, 20), (15, 25)]
THRESHOLDS  = np.linspace(0.05, 0.95, 19)

RANDOM_STATE = 26

# TRUE personalized: default = False (no semi/global)
USE_USER_ID_FEATURE = False

# threshold tuning:
# - True  => tune threshold terpisah per user (lebih fleksibel, tapi harus fair: pakai CV user tsb)
# - False => tune 1 threshold global (pooled across users) untuk semua user
TUNE_THRESHOLD_PER_USER = True

# Print detail per-user untuk baselines & model terpilih
PRINT_PER_USER_DETAILS = True


# =========================
# Print helpers (rapih, no styling berlebihan)
# =========================
def section(title: str):
    print("\n" + "=" * 80)
    print(title)
    print("=" * 80)

def kv(k, v):
    print(f"{k:<18}: {v}")

def safe_class_counts(y):
    y = np.asarray(y).astype(int)
    return {0: int((y == 0).sum()), 1: int((y == 1).sum())}

def print_per_user_breakdown(title, per_user_records, thr_info=None):
    print(f"\n{title}")
    for r in per_user_records:
        uid = r["uid"]
        y = np.asarray(r["y"]).astype(int)
        pred = np.asarray(r["pred"]).astype(int)
        acc = float(accuracy_score(y, pred))
        f1  = float(f1_score(y, pred, zero_division=0))
        dist = safe_class_counts(y)

        extra = ""
        if thr_info is not None:
            if isinstance(thr_info, dict) and uid in thr_info:
                extra = f" | thr={float(thr_info[uid]):.2f}"
            elif isinstance(thr_info, (float, int)):
                extra = f" | thr={float(thr_info):.2f}"

        print(f"uid={uid} | n={len(y):<3} | dist={dist} | acc={acc:.4f} | f1={f1:.4f}{extra}")


# =========================
# Helpers
# =========================
def eval_bin(y_true, y_pred):
    return {
        "acc": float(accuracy_score(y_true, y_pred)),
        "f1":  float(f1_score(y_true, y_pred, zero_division=0)),
    }

def tune_thr_from_proba(y_true, p_high, thresholds=THRESHOLDS):
    best_thr, best_f1 = None, -1.0
    for thr in thresholds:
        pred = (p_high >= thr).astype(int)
        f1 = float(f1_score(y_true, pred, zero_division=0))
        if f1 > best_f1:
            best_f1, best_thr = f1, thr
    return float(best_thr), float(best_f1)

def per_user_macro_metrics(per_user_records):
    accs, f1s = [], []
    for r in per_user_records:
        accs.append(accuracy_score(r["y"], r["pred"]))
        f1s.append(f1_score(r["y"], r["pred"], zero_division=0))
    return float(np.mean(accs)), float(np.mean(f1s))

def cv_folds_user(tp_df):
    folds = []
    for (v0, v1) in VAL_WINDOWS:
        if len(tp_df) < v1:
            continue
        tr = tp_df.iloc[:v0].copy()
        va = tp_df.iloc[v0:v1].copy()
        folds.append((tr, va))
    return folds

def min_class_count(y):
    vc = pd.Series(np.asarray(y)).value_counts()
    if len(vc) < 2:
        return 0
    return int(vc.min())


# =========================
# 1) LOAD + FEATURE ENGINEERING (no leak)
# =========================
section("1) LOAD + FEATURE ENGINEERING (NO-LEAK)")

df = pd.read_csv(DATA_PATH)
if "is_restored" not in df.columns:
    df["is_restored"] = 0
df["is_restored"] = df["is_restored"].fillna(0).astype(int)

if DATE_COL not in df.columns:
    raise KeyError(f"Kolom {DATE_COL} tidak ditemukan.")
if USER_COL not in df.columns:
    raise KeyError(f"Kolom {USER_COL} tidak ditemukan.")
if TARGET_COL not in df.columns:
    raise KeyError(f"Kolom {TARGET_COL} tidak ditemukan.")

df[DATE_COL] = pd.to_datetime(df[DATE_COL], errors="raise")
df = df.sort_values([USER_COL, DATE_COL]).reset_index(drop=True)

if not df[TARGET_COL].dropna().between(0, 2).all():
    raise ValueError(f"{TARGET_COL} harus berada pada range 0..2")

rows = []
for uid, g in df.groupby(USER_COL):
    g = g.sort_values(DATE_COL).reset_index(drop=True)

    g["dow"] = g[DATE_COL].dt.dayofweek.astype(int)
    g["is_weekend"] = (g["dow"] >= 5).astype(int)

    for k in range(1, WINDOW + 1):
        g[f"lag_sp_{k}"] = g[TARGET_COL].shift(k)

    sp_shift = g[TARGET_COL].shift(1)

    g["sp_mean"] = sp_shift.rolling(WINDOW).mean()
    g["sp_std"]  = sp_shift.rolling(WINDOW).std().fillna(0.0)
    g["sp_min"]  = sp_shift.rolling(WINDOW).min()
    g["sp_max"]  = sp_shift.rolling(WINDOW).max()

    g["count_high"] = (sp_shift >= 1).rolling(WINDOW).sum()
    g["count_low"]  = (sp_shift == 0).rolling(WINDOW).sum()

    high = (sp_shift >= 1).astype(int).fillna(0).astype(int).tolist()
    streak, cur = [], 0
    for v in high:
        cur = cur + 1 if v == 1 else 0
        streak.append(cur)
    g["streak_high"] = streak

    diff = (sp_shift != sp_shift.shift(1)).astype(int)
    g["transitions"] = diff.rolling(WINDOW).sum()

    rows.append(g)

feat = pd.concat(rows, ignore_index=True)
feat["y_bin"] = (feat[TARGET_COL] >= 1).astype(int)

feature_cols = (
    ["dow", "is_weekend"]
    + [f"lag_sp_{k}" for k in range(1, WINDOW + 1)]
    + [
        "sp_mean", "sp_std", "sp_min", "sp_max",
        "count_high", "count_low",
        "streak_high", "transitions",
    ]
)
if USE_USER_ID_FEATURE:
    feature_cols = [USER_COL] + feature_cols

feat = feat.dropna(subset=feature_cols + ["y_bin"]).reset_index(drop=True)
users = sorted(feat[USER_COL].unique().tolist())

kv("DATA_PATH", str(DATA_PATH))
kv("ROWS_FEAT", len(feat))
kv("USERS", users)
kv("DATE_RANGE", f"{feat[DATE_COL].min().date()} -> {feat[DATE_COL].max().date()}")
kv("WINDOW", WINDOW)
kv("TEST_LEN", TEST_LEN)
kv("FEATURES_COUNT", len(feature_cols))
kv("USE_USER_ID_FEATURE", USE_USER_ID_FEATURE)
kv("BINARY_DIST", feat["y_bin"].value_counts().to_dict())
kv("VAL_WINDOWS", VAL_WINDOWS)
kv("TUNE_THR_PER_USER", TUNE_THRESHOLD_PER_USER)


# =========================
# 2) SPLIT per user (time-based)
# =========================
section("2) SPLIT PER USER (TIME-BASED)")

per_user = {}
split_rows = []
for uid in users:
    g = feat[feat[USER_COL] == uid].sort_values(DATE_COL).reset_index(drop=True)
    n = len(g)
    test_start = n - TEST_LEN
    if test_start <= 10:
        raise ValueError(f"User {uid}: data terlalu sedikit untuk split (n={n}, TEST_LEN={TEST_LEN}).")

    tp = g.iloc[:test_start].copy()
    te = g.iloc[test_start:].copy()

    per_user[uid] = {"train_pool": tp, "test": te}

    split_rows.append({
        "uid": uid,
        "n_total": n,
        "n_train_pool": len(tp),
        "n_test": len(te),
        "train_pool_dist": safe_class_counts(tp["y_bin"].values),
        "test_dist": safe_class_counts(te["y_bin"].values),
    })

kv("TOTAL_TRAINPOOL", sum(r["n_train_pool"] for r in split_rows))
kv("TOTAL_TEST", sum(r["n_test"] for r in split_rows))
print("\nPER_USER_SPLIT:")
for r in split_rows:
    print(
        f"uid={r['uid']} | total={r['n_total']} | train_pool={r['n_train_pool']} dist={r['train_pool_dist']} "
        f"| test={r['n_test']} dist={r['test_dist']}"
    )


# =========================
# 3) BASELINE L1: Persistence
# =========================
section("3) BASELINE L1: PERSISTENCE (PER USER)")

persist_user_records = []
all_true, all_pred = [], []

for uid in users:
    te = per_user[uid]["test"]
    y = te["y_bin"].astype(int).values
    pred = (te["lag_sp_1"] >= 1).astype(int).values

    persist_user_records.append({"uid": uid, "y": y, "pred": pred})
    all_true.append(y)
    all_pred.append(pred)

y_all = np.concatenate(all_true)
pred_all = np.concatenate(all_pred)

persist_pooled = eval_bin(y_all, pred_all)
persist_macro_acc, persist_macro_f1 = per_user_macro_metrics(persist_user_records)

kv("TEST_POOLED_ACC", persist_pooled["acc"])
kv("TEST_POOLED_F1", persist_pooled["f1"])
kv("TEST_MACRO_ACC", persist_macro_acc)
kv("TEST_MACRO_F1", persist_macro_f1)

if PRINT_PER_USER_DETAILS:
    print_per_user_breakdown("PER-USER (Persistence) on TEST:", persist_user_records)


# =========================
# 4) BASELINE L2: Markov USER(prev_high, dow)
# =========================
section("4) BASELINE L2: MARKOV PER USER (prev_high, dow) + THR TUNING")

def train_markov_one_user(df_train):
    counts = np.zeros((2, 7, 2), dtype=int)  # prev(2) x dow(7) x y(2)
    prev = (df_train["lag_sp_1"] >= 1).astype(int).values
    dow  = (df_train["dow"]).astype(int).values
    yb   = (df_train["y_bin"]).astype(int).values
    for p, d, y in zip(prev, dow, yb):
        counts[p, d, y] += 1
    probs = (counts + 1) / (counts.sum(axis=2, keepdims=True) + 2)  # Laplace smoothing
    return probs

def markov_proba_user(probs, df_eval):
    prev = (df_eval["lag_sp_1"] >= 1).astype(int).values
    dow  = (df_eval["dow"]).astype(int).values
    return np.array([probs[p, d, 1] for p, d in zip(prev, dow)], dtype=float)

# pooled CV untuk threshold (konsisten & fair)
cv_true, cv_phigh = [], []
cv_fold_stats = []

for uid in users:
    tp = per_user[uid]["train_pool"]
    folds = cv_folds_user(tp)
    for (tr_df, va_df) in folds:
        probs = train_markov_one_user(tr_df)
        p = markov_proba_user(probs, va_df)
        cv_true.append(va_df["y_bin"].astype(int).values)
        cv_phigh.append(p)
        cv_fold_stats.append({"uid": uid, "tr_len": len(tr_df), "va_len": len(va_df)})

if len(cv_true) == 0:
    raise ValueError("Tidak ada CV fold yang valid. Kurangi VAL_WINDOWS / TEST_LEN / WINDOW.")

cv_true = np.concatenate(cv_true)
cv_phigh = np.concatenate(cv_phigh)

thr_mk, cv_f1_mk = tune_thr_from_proba(cv_true, cv_phigh)

mk_models = {}
markov_user_records = []
all_true, all_pred = [], []

for uid in users:
    tp = per_user[uid]["train_pool"]
    te = per_user[uid]["test"]

    probs = train_markov_one_user(tp)
    mk_models[uid] = probs

    p = markov_proba_user(probs, te)
    pred = (p >= thr_mk).astype(int)
    y = te["y_bin"].astype(int).values

    markov_user_records.append({"uid": uid, "y": y, "pred": pred})
    all_true.append(y)
    all_pred.append(pred)

y_all = np.concatenate(all_true)
pred_all = np.concatenate(all_pred)

markov_pooled = eval_bin(y_all, pred_all)
markov_macro_acc, markov_macro_f1 = per_user_macro_metrics(markov_user_records)

kv("CV_FOLDS_TOTAL", len(cv_fold_stats))
kv("CV_POOLED_DIST", safe_class_counts(cv_true))
kv("BEST_THR_MARKOV", thr_mk)
kv("CV_POOLED_F1", cv_f1_mk)
kv("TEST_POOLED_ACC", markov_pooled["acc"])
kv("TEST_POOLED_F1", markov_pooled["f1"])
kv("TEST_MACRO_ACC", markov_macro_acc)
kv("TEST_MACRO_F1", markov_macro_f1)

if PRINT_PER_USER_DETAILS:
    print_per_user_breakdown("PER-USER (Markov) on TEST:", markov_user_records, thr_info=thr_mk)


# =========================
# 5) Preprocess (for ML)
# =========================
section("5) PREPROCESS (FOR ML)")

cat_cols = ["dow", "is_weekend"]
if USE_USER_ID_FEATURE:
    cat_cols = [USER_COL] + cat_cols

num_cols = [c for c in feature_cols if c not in cat_cols]

preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
        ("num", Pipeline([("imp", SimpleImputer(strategy="median"))]), num_cols),
    ],
    remainder="drop",
)

kv("CAT_COLS", cat_cols)
kv("NUM_COLS_COUNT", len(num_cols))


# =========================
# 6) Candidate models (non-SVM) + SVM SAFE config
# =========================
section("6) CANDIDATE MODELS")

# Bagging compatibility (estimator vs base_estimator)
try:
    bag_base = BaggingClassifier(
        estimator=DecisionTreeClassifier(random_state=RANDOM_STATE),
        random_state=RANDOM_STATE,
        n_jobs=1
    )
    BAG_ESTIMATOR_PARAM = "clf__estimator__"
except TypeError:
    bag_base = BaggingClassifier(
        base_estimator=DecisionTreeClassifier(random_state=RANDOM_STATE),
        random_state=RANDOM_STATE,
        n_jobs=1
    )
    BAG_ESTIMATOR_PARAM = "clf__base_estimator__"

CANDIDATES = {
    "LogReg": (
        LogisticRegression(max_iter=5000, class_weight="balanced", random_state=RANDOM_STATE),
        {"clf__C": [0.03, 0.1, 0.3, 1.0, 3.0], "clf__solver": ["liblinear"]}
    ),
    "DecisionTree": (
        DecisionTreeClassifier(class_weight="balanced", random_state=RANDOM_STATE),
        {"clf__max_depth": [2, 3, 4, 6, None], "clf__min_samples_leaf": [1, 2, 4, 8]}
    ),
    "RandomForest": (
        RandomForestClassifier(class_weight="balanced", random_state=RANDOM_STATE, n_jobs=1),
        {"clf__n_estimators": [200, 400, 800], "clf__max_depth": [None, 6, 10],
         "clf__min_samples_leaf": [1, 2, 4], "clf__max_features": ["sqrt"]}
    ),
    "ExtraTrees": (
        ExtraTreesClassifier(class_weight="balanced", random_state=RANDOM_STATE, n_jobs=1),
        {"clf__n_estimators": [200, 400, 800], "clf__max_depth": [None, 6, 10],
         "clf__min_samples_leaf": [1, 2, 4], "clf__max_features": ["sqrt"]}
    ),
    "HistGB": (
        HistGradientBoostingClassifier(random_state=RANDOM_STATE),
        {"clf__learning_rate": [0.03, 0.05, 0.1], "clf__max_depth": [2, 3],
         "clf__max_leaf_nodes": [15, 31, 63]}
    ),
    "GradBoost": (
        GradientBoostingClassifier(random_state=RANDOM_STATE),
        {"clf__learning_rate": [0.03, 0.05, 0.1], "clf__n_estimators": [100, 200, 400],
         "clf__max_depth": [2, 3]}
    ),
    "AdaBoost": (
        AdaBoostClassifier(random_state=RANDOM_STATE),
        {"clf__learning_rate": [0.03, 0.05, 0.1, 0.3], "clf__n_estimators": [50, 100, 200, 400]}
    ),
    "BaggingTree": (
        bag_base,
        {"clf__n_estimators": [50, 100, 200],
         f"{BAG_ESTIMATOR_PARAM}max_depth": [2, 3, 4, None],
         f"{BAG_ESTIMATOR_PARAM}min_samples_leaf": [1, 2, 4]}
    ),
}

SVM_NAME = "LinearSVC_Calibrated_SAFE"
SVM_GRID = {"C": [0.03, 0.1, 0.3, 1.0, 3.0]}

kv("MODELS", list(CANDIDATES.keys()) + [SVM_NAME])
kv("THRESHOLDS_COUNT", len(THRESHOLDS))


# =========================
# 7) Tuning utilities (threshold global or per-user)
# =========================
section("7) TUNING UTILITIES")

def tune_global_thr_pooled_over_all_users(pipe, params):
    y_list, p_list = [], []
    for uid in users:
        tp = per_user[uid]["train_pool"]
        folds = cv_folds_user(tp)
        for tr_df, va_df in folds:
            ytr = tr_df["y_bin"].astype(int).values
            if len(np.unique(ytr)) < 2:
                continue
            pipe.set_params(**params)
            pipe.fit(tr_df[feature_cols], ytr)
            p = pipe.predict_proba(va_df[feature_cols])[:, 1]
            y_list.append(va_df["y_bin"].astype(int).values)
            p_list.append(p)

    if len(y_list) == 0:
        return None, None

    y_all = np.concatenate(y_list)
    p_all = np.concatenate(p_list)
    thr, cv_f1 = tune_thr_from_proba(y_all, p_all)
    return float(thr), float(cv_f1)

def tune_per_user_thr(pipe, params):
    thr_by_user = {}
    f1s = []
    for uid in users:
        tp = per_user[uid]["train_pool"]
        folds = cv_folds_user(tp)
        if len(folds) == 0:
            return None, None

        y_list, p_list = [], []
        for tr_df, va_df in folds:
            ytr = tr_df["y_bin"].astype(int).values
            if len(np.unique(ytr)) < 2:
                continue
            pipe.set_params(**params)
            pipe.fit(tr_df[feature_cols], ytr)
            p = pipe.predict_proba(va_df[feature_cols])[:, 1]
            y_list.append(va_df["y_bin"].astype(int).values)
            p_list.append(p)

        if len(y_list) == 0:
            return None, None

        y_u = np.concatenate(y_list)
        p_u = np.concatenate(p_list)
        thr_u, f1_u = tune_thr_from_proba(y_u, p_u)
        thr_by_user[uid] = float(thr_u)
        f1s.append(float(f1_u))

    return thr_by_user, float(np.mean(f1s))

def eval_personalized_models(models_by_user, thr_by_user_or_scalar):
    per_user_records = []
    all_true, all_pred = [], []

    for uid in users:
        te = per_user[uid]["test"]
        y = te["y_bin"].astype(int).values

        pipe = models_by_user[uid]
        p = pipe.predict_proba(te[feature_cols])[:, 1]
        thr = thr_by_user_or_scalar[uid] if isinstance(thr_by_user_or_scalar, dict) else float(thr_by_user_or_scalar)
        pred = (p >= thr).astype(int)

        per_user_records.append({"uid": uid, "y": y, "pred": pred})
        all_true.append(y)
        all_pred.append(pred)

    y_all = np.concatenate(all_true)
    pred_all = np.concatenate(all_pred)

    pooled = eval_bin(y_all, pred_all)
    macro_acc, macro_f1 = per_user_macro_metrics(per_user_records)
    macro = {"acc": float(macro_acc), "f1": float(macro_f1)}
    return pooled, macro, per_user_records


# =========================
# 8) TRAIN + TUNE all non-SVM candidates
# =========================
section("8) PERSONALIZED ML: TRAIN + TUNE (NON-SVM)")

rows = []

for name, (clf, grid) in CANDIDATES.items():
    best = None

    for params in ParameterGrid(grid):
        pipe = Pipeline([("prep", preprocess), ("clf", clf)])

        if TUNE_THRESHOLD_PER_USER:
            thr_obj, cv_score = tune_per_user_thr(pipe, params)
        else:
            thr_obj, cv_score = tune_global_thr_pooled_over_all_users(pipe, params)

        if thr_obj is None:
            continue

        if (best is None) or (cv_score > best["cv_score"]):
            best = {"params": dict(params), "thr_obj": thr_obj, "cv_score": float(cv_score)}

    if best is None:
        print(f"SKIP_MODEL         : {name} (no valid params/folds)")
        continue

    # train final per-user
    models_by_user = {}
    ok = True
    for uid in users:
        tp = per_user[uid]["train_pool"]
        ytr = tp["y_bin"].astype(int).values
        if len(np.unique(ytr)) < 2:
            ok = False
            break

        pipe = Pipeline([("prep", preprocess), ("clf", clf)])
        pipe.set_params(**best["params"])
        pipe.fit(tp[feature_cols], ytr)
        models_by_user[uid] = pipe

    if not ok:
        print(f"SKIP_MODEL         : {name} (some user train_pool has single class)")
        continue

    pooled, macro, user_records = eval_personalized_models(models_by_user, best["thr_obj"])

    rows.append({
        "model": name,
        "cv_score": float(best["cv_score"]),
        "thr_obj": best["thr_obj"],
        "test_pooled_f1": float(pooled["f1"]),
        "test_pooled_acc": float(pooled["acc"]),
        "test_macro_f1": float(macro["f1"]),
        "test_macro_acc": float(macro["acc"]),
        "params": dict(best["params"]),
        "models_by_user": models_by_user,
        "test_user_records": user_records,
    })

    thr_desc = "per-user" if isinstance(best["thr_obj"], dict) else f"{best['thr_obj']:.2f}"
    print(f"MODEL              : {name}")
    kv("  CV_SCORE", best["cv_score"])
    kv("  THRESHOLD", thr_desc)
    kv("  TEST_POOLED_F1", pooled["f1"])
    kv("  TEST_POOLED_ACC", pooled["acc"])
    kv("  TEST_MACRO_F1", macro["f1"])
    kv("  TEST_MACRO_ACC", macro["acc"])
    kv("  PARAMS", best["params"])


# =========================
# 9) SVM Calibrated SAFE (FIXED)
# =========================
section("9) SVM CALIBRATED SAFE (FIXED)")

def make_calibrator(base, cv_k):
    try:
        return CalibratedClassifierCV(estimator=base, method="sigmoid", cv=cv_k)
    except TypeError:
        return CalibratedClassifierCV(base_estimator=base, method="sigmoid", cv=cv_k)

def svm_fit_predict_proba(tr_X, tr_y, va_X, C, cv_max=3):
    mcc = min_class_count(tr_y)
    cv_k = int(min(cv_max, mcc))
    if cv_k < 2:
        return None, cv_k
    base = LinearSVC(class_weight="balanced", random_state=RANDOM_STATE, C=float(C))
    calib = make_calibrator(base, cv_k=cv_k)
    pipe = Pipeline([("prep", preprocess), ("clf", calib)])
    pipe.fit(tr_X, tr_y)
    return pipe.predict_proba(va_X)[:, 1], cv_k

# 1) Check feasibility for all users in final training
svm_feasible_all_users = True
svm_feasible_detail = []
for uid in users:
    y_tp = per_user[uid]["train_pool"]["y_bin"].astype(int).values
    mcc = min_class_count(y_tp)
    svm_feasible_detail.append({"uid": uid, "min_class_count_trainpool": mcc})
    if mcc < 2:
        svm_feasible_all_users = False

kv("SVM_FEASIBLE_ALL_USERS", svm_feasible_all_users)
print("SVM_FEASIBLE_DETAIL:")
for r in svm_feasible_detail:
    print(f"uid={r['uid']} | min_class_count_trainpool={r['min_class_count_trainpool']}")

if svm_feasible_all_users:
    best = None

    for C in SVM_GRID["C"]:
        if TUNE_THRESHOLD_PER_USER:
            thr_by_user = {}
            per_user_cv_scores = []
            all_users_ok = True
            users_valid = 0

            for uid in users:
                tp = per_user[uid]["train_pool"]
                folds = cv_folds_user(tp)

                y_list_u, p_list_u = [], []

                for (tr_df, va_df) in folds:
                    tr_y = tr_df["y_bin"].astype(int).values
                    p, cv_k = svm_fit_predict_proba(tr_df[feature_cols], tr_y, va_df[feature_cols], C=C, cv_max=3)
                    if p is None:
                        continue
                    y_list_u.append(va_df["y_bin"].astype(int).values)
                    p_list_u.append(p)

                if len(y_list_u) == 0:
                    all_users_ok = False
                    break

                y_u = np.concatenate(y_list_u)
                p_u = np.concatenate(p_list_u)
                thr_u, f1_u = tune_thr_from_proba(y_u, p_u)
                thr_by_user[uid] = float(thr_u)
                per_user_cv_scores.append(float(f1_u))
                users_valid += 1

            if (not all_users_ok) or (users_valid < len(users)):
                continue

            cv_score = float(np.mean(per_user_cv_scores))
            thr_obj = thr_by_user

        else:
            y_list, p_list = [], []
            for uid in users:
                tp = per_user[uid]["train_pool"]
                folds = cv_folds_user(tp)
                for (tr_df, va_df) in folds:
                    tr_y = tr_df["y_bin"].astype(int).values
                    p, cv_k = svm_fit_predict_proba(tr_df[feature_cols], tr_y, va_df[feature_cols], C=C, cv_max=3)
                    if p is None:
                        continue
                    y_list.append(va_df["y_bin"].astype(int).values)
                    p_list.append(p)

            if len(y_list) == 0:
                continue

            y_all = np.concatenate(y_list)
            p_all = np.concatenate(p_list)
            thr_obj, cv_score = tune_thr_from_proba(y_all, p_all)

        if (best is None) or (cv_score > best["cv_score"]):
            best = {"C": float(C), "thr_obj": thr_obj, "cv_score": float(cv_score)}

    if best is None:
        print(f"SKIP_MODEL         : {SVM_NAME} (no valid C setting across all users/folds)")
    else:
        # 2) Final training per user (cv adaptif dari train_pool)
        models_by_user = {}
        ok = True
        final_cv_by_user = {}

        for uid in users:
            tp = per_user[uid]["train_pool"]
            tr_y = tp["y_bin"].astype(int).values
            mcc = min_class_count(tr_y)
            cv_k = int(min(3, mcc))
            if cv_k < 2:
                ok = False
                break

            base = LinearSVC(class_weight="balanced", random_state=RANDOM_STATE, C=float(best["C"]))
            calib = make_calibrator(base, cv_k=cv_k)
            pipe = Pipeline([("prep", preprocess), ("clf", calib)])
            pipe.fit(tp[feature_cols], tr_y)

            models_by_user[uid] = pipe
            final_cv_by_user[uid] = cv_k

        if not ok:
            print(f"SKIP_MODEL         : {SVM_NAME} (final training not feasible for all users)")
        else:
            pooled, macro, user_records = eval_personalized_models(models_by_user, best["thr_obj"])
            rows.append({
                "model": SVM_NAME,
                "cv_score": float(best["cv_score"]),
                "thr_obj": best["thr_obj"],
                "test_pooled_f1": float(pooled["f1"]),
                "test_pooled_acc": float(pooled["acc"]),
                "test_macro_f1": float(macro["f1"]),
                "test_macro_acc": float(macro["acc"]),
                "params": {"C": float(best["C"]), "calibration_cv": f"adaptive<=3 (per user), {final_cv_by_user}"},
                "models_by_user": models_by_user,
                "test_user_records": user_records,
            })

            thr_desc = "per-user" if isinstance(best["thr_obj"], dict) else f"{best['thr_obj']:.2f}"
            print(f"MODEL              : {SVM_NAME}")
            kv("  CV_SCORE", best["cv_score"])
            kv("  THRESHOLD", thr_desc)
            kv("  TEST_POOLED_F1", pooled["f1"])
            kv("  TEST_POOLED_ACC", pooled["acc"])
            kv("  TEST_MACRO_F1", macro["f1"])
            kv("  TEST_MACRO_ACC", macro["acc"])
            kv("  PARAMS", {"C": best["C"], "final_cv_by_user": final_cv_by_user})
else:
    print(f"SKIP_MODEL         : {SVM_NAME} (some user train_pool has single class)")


# =========================
# 10) LEADERBOARD + SELECT BEST vs Markov
# =========================
section("10) LEADERBOARD + SELECT BEST")

print("BASELINES:")
print(f"  Baseline-Persist | TEST pooled: acc={persist_pooled['acc']:.4f}, f1={persist_pooled['f1']:.4f} | "
      f"macro(user): acc={persist_macro_acc:.4f}, f1={persist_macro_f1:.4f}")
print(f"  Baseline-Markov  | CV pooled: f1={cv_f1_mk:.4f}, thr={thr_mk:.2f} | "
      f"TEST pooled: acc={markov_pooled['acc']:.4f}, f1={markov_pooled['f1']:.4f} | "
      f"macro(user): acc={markov_macro_acc:.4f}, f1={markov_macro_f1:.4f}")

rows_sorted = sorted(rows, key=lambda r: r["test_pooled_f1"], reverse=True)

print("\nCANDIDATES:")
if len(rows_sorted) == 0:
    print("  (no ML candidates succeeded)")
else:
    for r in rows_sorted:
        thr_desc = "per-user" if isinstance(r["thr_obj"], dict) else f"{r['thr_obj']:.2f}"
        print(
            f"  {r['model']:<24} | CV={r['cv_score']:.4f} | thr={thr_desc:<8} | "
            f"TEST pooled: acc={r['test_pooled_acc']:.4f}, f1={r['test_pooled_f1']:.4f} | "
            f"macro(user): acc={r['test_macro_acc']:.4f}, f1={r['test_macro_f1']:.4f} | params={r['params']}"
        )

best_name = "MarkovUser"
best_obj = {"type": "markov_user", "thr": float(thr_mk), "probs_by_user": mk_models}
best_test_pooled_f1 = float(markov_pooled["f1"])
best_user_records = markov_user_records
best_thr_info = thr_mk

if len(rows_sorted) > 0 and float(rows_sorted[0]["test_pooled_f1"]) > best_test_pooled_f1:
    top = rows_sorted[0]
    best_name = top["model"]
    best_obj = {
        "type": "personalized_sklearn",
        "models_by_user": top["models_by_user"],
        "thr": top["thr_obj"],
        "meta": {"tune_threshold_per_user": bool(TUNE_THRESHOLD_PER_USER)},
    }
    best_user_records = top["test_user_records"]
    best_thr_info = top["thr_obj"]

print("\nSELECTED_BEST      :", best_name)
if best_name == "MarkovUser":
    print("SELECT_REASON      : Markov baseline remains best on TEST pooled F1 for this dataset.")

if PRINT_PER_USER_DETAILS:
    print_per_user_breakdown(f"PER-USER (SELECTED_BEST={best_name}) on TEST:", best_user_records, thr_info=best_thr_info)


# =========================
# 11) SAVE ARTIFACT
# =========================
section("11) SAVE ARTIFACT")

MODEL_OUT.parent.mkdir(parents=True, exist_ok=True)
joblib.dump(
    {
        "best_name": best_name,
        "artifact": best_obj,
        "meta": {
            "target": "y_bin = (stress_level_pred>=1)",
            "date_col": DATE_COL,
            "user_col": USER_COL,
            "target_col": TARGET_COL,
            "window": WINDOW,
            "test_len": TEST_LEN,
            "val_windows": VAL_WINDOWS,
            "thresholds": THRESHOLDS.tolist(),
            "users": users,
            "baseline_l1": "persistence(per-user)",
            "baseline_l2": "markov_user(prev_high, dow)",
            "use_user_id_feature": USE_USER_ID_FEATURE,
            "tune_threshold_per_user": TUNE_THRESHOLD_PER_USER,
            "random_state": RANDOM_STATE,
            "feature_cols": feature_cols,
        }
    },
    MODEL_OUT
)

kv("SAVED_TO", str(MODEL_OUT))
kv("BEST_NAME", best_name)



1) LOAD + FEATURE ENGINEERING (NO-LEAK)
DATA_PATH         : ..\datasets\global_dataset_pred.csv
ROWS_FEAT         : 260
USERS             : [1, 2, 3, 4, 5]
DATE_RANGE        : 2025-11-24 -> 2026-01-14
WINDOW            : 3
TEST_LEN          : 12
FEATURES_COUNT    : 13
USE_USER_ID_FEATURE: False
BINARY_DIST       : {1: 146, 0: 114}
VAL_WINDOWS       : [(10, 20), (15, 25)]
TUNE_THR_PER_USER : True

2) SPLIT PER USER (TIME-BASED)
TOTAL_TRAINPOOL   : 200
TOTAL_TEST        : 60

PER_USER_SPLIT:
uid=1 | total=52 | train_pool=40 dist={0: 18, 1: 22} | test=12 dist={0: 3, 1: 9}
uid=2 | total=52 | train_pool=40 dist={0: 16, 1: 24} | test=12 dist={0: 6, 1: 6}
uid=3 | total=52 | train_pool=40 dist={0: 25, 1: 15} | test=12 dist={0: 7, 1: 5}
uid=4 | total=52 | train_pool=40 dist={0: 15, 1: 25} | test=12 dist={0: 2, 1: 10}
uid=5 | total=52 | train_pool=40 dist={0: 18, 1: 22} | test=12 dist={0: 4, 1: 8}

3) BASELINE L1: PERSISTENCE (PER USER)
TEST_POOLED_ACC   : 0.7166666666666667
TEST_POOLED_F1    :